In [1]:
%pip install nltk
%pip install keras
%pip install tensorflow
%pip install tensorflow_hub
%pip install transformers
%pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss


2023-10-13 19:20:20.975910: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 19:20:23.607929: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-13 19:20:23.607977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-13 19:20:23.611847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-13 19:20:24.626276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 19:20:24.627502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
 # Load your dataset from the DataFrame
df = pd.read_csv('/workspaces/CONAN/Multitarget-CONAN/Multitarget-CONAN.csv')
df.head()

INDEX                                        HATE_SPEECH  \
0      0  Maybe the UN could talk to those asian and afr...   
1      1  ‘Juice (Jews) are worse that nukes.’ Caption t...   
2      2  Foreigners on UK benefits leaps 41% in 5 years...   
3      3  Jews cannot be patriots, since their allegianc...   
4      4  Israel, Zionism and Jewish Supremacist control...   

                                   COUNTER_NARRATIVE    TARGET VERSION  
0  You're right, ocean pollution is one of the ma...     other      V1  
1                        You'd rather be nuked then?      JEWS      V1  
2  You seem to be worried about government expend...  MIGRANTS      V1  
3  You can have parents and grandparents born els...      JEWS      V1  
4  Wouldn't you say its colonialism from many dif...      JEWS      V1

In [4]:
# Assuming 'df' is your DataFrame
df['combined_text'] = df['HATE_SPEECH'] + " " + df['COUNTER_NARRATIVE']
df['combined_text'][1]

with open('gpt2_training_data.txt', 'w') as file:
    for text in df['combined_text']:
        file.write(text + '\n')


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Initialize the GPT-2 tokenizer and model
model_name = "gpt2"  # You can choose another variant if needed
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load your preprocessed dataset
dataset_path = "/workspaces/CONAN/DSML Research Project/gpt2_training_data.txt"
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=dataset_path,
    block_size=128,  # Adjust the block size as needed
)

# Define a data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We're doing language modeling, not masked language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",  # Output directory for checkpoints and logs
    overwrite_output_dir=True,
    per_device_train_batch_size=2,  # Adjust batch size as needed
    num_train_epochs=3,  # Adjust the number of training epochs
    save_steps=10_000,  # Adjust checkpoint save frequency
    save_total_limit=2,  # Limit the number of checkpoints saved
)

# Initialize the Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Start fine-tuning
trainer.train()

# Save the model
trainer.save_model()

# Optionally, you can save the tokenizer and config as well
tokenizer.save_pretrained("./output")
model.config.save_pretrained("./output")


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.python/current/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


In [8]:
# Define a prompt with hate speech

prompt = "I hate Muslims because they are terrorists."

# Generate a counter-narrative response
input_ids = tokenizer.encode(prompt, return_tensors="pt")
response_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode and print the generated counter-narrative
counter_narrative = tokenizer.decode(response_ids[0], skip_special_tokens=True)
print("Counter-Narrative:", counter_narrative)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Counter-Narrative: I hate Muslims because they are terrorists. They are not terrorists. They are just people who have different beliefs and beliefs.
Muslims are a problem in my opinion. They are not welcome here. They are not like us. They are not like us
